In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r'C:\Users\b.Yuva Shree\Downloads\archive (14)\movie_tvshow_description.csv')

In [3]:
df.head()

,url,description,genres,title
0,https://www.metacritic.com/movie/the-informer/,Honorably discharged Special Ops soldier Pete ...,"['Action', 'Crime', 'Drama', 'Thriller', '', '...","7,921. The Informer\nNOV 6, 2020  \nHonorabl..."
1,https://www.metacritic.com/movie/reign-over-me/,Former college roommates Charlie Fineman (Sand...,"['Drama', '', 'Drama']","7,922.\nMAR 23, 2007\nFormer college roommates..."
2,https://www.metacritic.com/movie/triple-frontier/,A group of former Special Forces operatives (B...,"['Action', 'Thriller', '', '', 'Action', 'Thri...","7,923.\nMAR 6, 2019  \nA group of former Spe..."
3,https://www.metacritic.com/movie/somewhere-in-...,Leo Russo (Ray Romano) lives a simple life in ...,"['Comedy', 'Drama', 'Sport', '', '', '', 'Come...","7,924.\nAPR 21, 2023  \nLeo Russo (Ray Roman..."
4,https://www.metacritic.com/movie/the-jeffrey-d...,In the summer of 1991 Jeffrey Dahmer was arres...,"['Documentary', 'Biography', 'Crime', 'Drama',...","7,925. The Jeffrey Dahmer Files\nFEB 15, 2013 ..."


In [4]:
df.tail()

,url,description,genres,title
21864,https://www.metacritic.com/tv/highway-to-heaven/,Highway to Heaven was Micheal Landon's second ...,"['Drama', 'Fantasy', '', '', 'Drama', 'Fantasy']",Martin
21865,https://www.metacritic.com/tv/punky-brewster/,"""Every time I turn around, I see the girl who ...","['Comedy', 'Family', '', '', 'Comedy', 'Family']",2000 Malibu Road
21866,https://www.metacritic.com/tv/miami-vice/,Miami Vice was one of the most innovative and ...,"['Action', 'Crime', 'Drama', 'Mystery', 'Thril...",The Larry Sanders Show
21867,https://www.metacritic.com/tv/danger-mouse/,Danger Mouse is about a cartoon mouse being a ...,"['Animation', 'Action', 'Adventure', 'Comedy',...",Are You Afraid of the Dark?
21868,https://www.metacritic.com/tv/double-trouble/,The show centers around twin sisters Allison a...,"['Comedy', '', 'Comedy']",Melrose Place (1992)


In [5]:
df.isnull().sum()

url            0
description    0
genres         0
title          0
dtype: int64

In [6]:
df.columns

Index(['url', 'description', 'genres', 'title'], dtype='object')

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\b.Yuva
[nltk_data]     Shree\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
des=df['description'].tolist()

In [10]:
des

["Honorably discharged Special Ops soldier Pete Koslow's (Joel Kinnaman) world is turned upside-down when he is jailed after a fight to protect his wife (Ana de Armas). He's given a chance for early release by becoming an informant for the FBI (Rosamund Pike and Clive Owen) and using his covert skills in an operation to take down The Gener...",
 'Former college roommates Charlie Fineman (Sandler) and Alan Johnson (Cheadle) meet up again by chance on a Manhattan street corner. Five years after losing his family on 9/11, Charlie has retreated from his life, and Alan is stunned to see the changes in his formerly gregarious friend. At the same time, Alan who should be enjoying his be...',
 'A group of former Special Forces operatives (Ben Affleck, Oscar Isaac, Charlie Hunnam, Garrett Hedlund and Pedro Pascal) reunite to plan a heist in a sparsely populated multi-border zone of South America. For the first time in their prestigious careers these unsung heroes undertake this dangerous missio

In [11]:
corpus = []
ps = PorterStemmer()
for i in range(0,df.shape[0]):
    message = re.sub(pattern='[^a-zA-Z]', repl=' ', string=df.description[i]) #Cleaning special character from the message
    message = message.lower() #Converting the entire message into lower case
    words = message.split() # Tokenizing the review by words
    words = [word for word in words if word not in set(stopwords.words('english'))] #Removing the stop words
    words = [ps.stem(word) for word in words] #Stemming the words
    message = ' '.join(words) #Joining the stemmed words
    corpus.append(message) #Building a corpus of messages

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [13]:
y = pd.get_dummies(df['genres'])
y = y.iloc[:, 1].values

In [14]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [15]:
df.drop('url', axis=1, inplace=True)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

In [18]:
best_accuracy = 0.0
alpha_val = 0.0
for i in np.arange(0.0,1.1,0.1):
    temp_classifier = MultinomialNB(alpha=i)
    temp_classifier.fit(X_train, y_train)
    temp_y_pred = temp_classifier.predict(X_test)
    score = accuracy_score(y_test, temp_y_pred)
    print("Accuracy score for alpha={} is: {}%".format(round(i,1), round(score*100,2)))
    if score>best_accuracy:
        best_accuracy = score
        alpha_val = i
print('--------------------------------------------')
print('The best accuracy is {}% with alpha value as {}'.format(round(best_accuracy*100, 2), round(alpha_val,1)))

C:\Users\b.Yuva Shree\AppData\Roaming\Python\Python311\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
C:\Users\b.Yuva Shree\AppData\Roaming\Python\Python311\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


Accuracy score for alpha=0.0 is: 99.93%
Accuracy score for alpha=0.1 is: 99.79%
Accuracy score for alpha=0.2 is: 99.84%
Accuracy score for alpha=0.3 is: 99.84%
Accuracy score for alpha=0.4 is: 99.84%
Accuracy score for alpha=0.5 is: 99.86%
Accuracy score for alpha=0.6 is: 99.86%
Accuracy score for alpha=0.7 is: 99.86%
Accuracy score for alpha=0.8 is: 99.86%
Accuracy score for alpha=0.9 is: 99.86%
Accuracy score for alpha=1.0 is: 99.86%
--------------------------------------------
The best accuracy is 99.93% with alpha value as 0.0


In [19]:
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train)

MultinomialNB(alpha=0.1)

In [20]:
y_pred = classifier.predict(X_test)

In [21]:
print("Predictions vs Actual Labels:")
for pred, actual in zip(y_pred[:10], y_test[:10]):
    print("Predicted:", pred, "| Actual:", actual)

Predictions vs Actual Labels:
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0


In [22]:
acc_s = accuracy_score(y_test, y_pred)*100

In [23]:
print("Accuracy Score {} %".format(round(acc_s,2)))

Accuracy Score 99.79 %


In [47]:
from sklearn.linear_model import LogisticRegression

In [50]:
log_model=LogisticRegression(random_state=0, max_iter=2000)

In [51]:
log_model.fit(X_train, y_train)

LogisticRegression(max_iter=2000, random_state=0)

In [52]:
y_pred=log_model.predict(X_test)

In [53]:
print("Predictions vs Actual Labels:")
for pred, actual in zip(y_pred[:10], y_test[:10]):
    print("Predicted:", pred, "| Actual:", actual)

Predictions vs Actual Labels:
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0


In [54]:
acc_s = accuracy_score(y_test, y_pred)*100

In [55]:
print("Accuracy Score {} %".format(round(acc_s,2)))

Accuracy Score 99.93 %


In [56]:
from sklearn.svm import SVC

In [57]:
svm_model=SVC(kernel="linear")

In [58]:
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [59]:
y_pred1=log_model.predict(X_test)

In [60]:
print("Predictions vs Actual Labels:")
for pred, actual in zip(y_pred1[:10], y_test[:10]):
    print("Predicted:", pred, "| Actual:", actual)

Predictions vs Actual Labels:
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0
Predicted: 0 | Actual: 0


In [61]:
acc_s = accuracy_score(y_test, y_pred)*100

In [62]:
print("Accuracy Score {} %".format(round(acc_s,2)))

Accuracy Score 99.93 %
